In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [15]:
connection = sqlite3.connect("./Data/db2assign2.db")
cursor = connection.cursor()

In [16]:
# Read in csv data
pd.read_csv('./Data/Department_Information.csv').to_sql("deptInfo", connection, if_exists='replace', index=True)
pd.read_csv('./Data/Employee_Information.csv').to_sql("employInfo", connection, if_exists='replace', index=True)
pd.read_csv('./Data/Student_Counceling_Information.csv').to_sql("studentCounselling", connection, if_exists='replace', index=True)
pd.read_csv('./Data/Student_Performance_Data.csv').to_sql("studentPerformance", connection, if_exists='replace', index=True)




209610

In [17]:
df = pd.read_sql_query("SELECT * FROM deptInfo", connection)